In [1]:
import pandas as pd, numpy as np, tabel, sys, io

from bokeh.plotting import figure, show, save
from bokeh.io import output_notebook
from bokeh.layouts import row, column
output_notebook()

Loading BokehJS ...

In [2]:
#CSV file can be downloaded from >https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=259

filename = "550482682_T_T100D_SEGMENT_US_CARRIER_ONLY.csv"

data_df = pd.read_csv(filename)
data_tabel = tabel.read_tabel(filename)

In [3]:
print('Dataset variables: '+(',  '.join(data_tabel.columns)))

Dataset variables: DEPARTURES_PERFORMED,  PAYLOAD,  SEATS,  PASSENGERS,  FREIGHT,  MAIL,  DISTANCE,  AIR_TIME,  ORIGIN_AIRPORT_ID,  ORIGIN,  ORIGIN_CITY_NAME,  ORIGIN_STATE_ABR,  ORIGIN_STATE_NM,  DEST_AIRPORT_ID,  DEST,  DEST_CITY_NAME,  DEST_STATE_ABR,  DEST_STATE_NM,  AIRCRAFT_TYPE,  AIRCRAFT_CONFIG,  YEAR,  MONTH


In [32]:
var_chosen = input("Which of the listed variables would you like to explore? ")

if var_chosen.upper() in data_tabel.columns:
    print("Okay! Generating insights for %s and variables similar..." %var_chosen.upper())
else:
    var_chosen = input("Error: variable chosen is not one of the listed options. Please be sure to match spelling. ")
    if var_chosen.upper() in data_tabel.columns:
        print("Okay! Generating insights for %s and variables similar..." %var_chosen.upper())
    else:
        sys.exit("Sorry, your input still doesn't match with any listed variables. Please re-run the program to try again.")

Which of the listed variables would you like to explore? seats
Okay! Generating insights for SEATS and variables similar...


In [33]:
total_dep_perf = sum(data_tabel['DEPARTURES_PERFORMED'])
month_weights = []

for i in range(max(data_tabel['MONTH'])):
    month = i+1
    month_dep = 0

    for row in range(len(data_tabel)): 
        if data_tabel[row, 'MONTH'] == month:
            month_dep += data_tabel[row, 'DEPARTURES_PERFORMED']

    month_weight = month_dep/total_dep_perf
    month_weights.append(month_weight)

In [34]:
if var_chosen.upper() == 'DEPARTURES_PERFORMED' or var_chosen.upper() == 'PASSENGERS' or var_chosen.upper() == 'SEATS':

    month_departures_perf_100k = []
    month_passengers = []
    month_passengers_100k = []
    month_fullness = []

    for i in range(max(data_tabel['MONTH'])):
        month = i+1
        month_dep = 0
        month_pass = 0 

        for row in range(len(data_tabel)): 
            if data_tabel[row, 'MONTH'] == month:
                month_dep += data_tabel[row, 'DEPARTURES_PERFORMED']
                month_pass += data_tabel[row, 'PASSENGERS']

        month_departures_perf_100k.append(month_dep/100000)
        month_passengers.append(month_pass)
        month_passengers_100k.append(month_pass/100000)


    for i in range(max(data_tabel['MONTH'])):
        month = i+1
        tot_month_fullness = 0

        for row in range(len(data_tabel)): 
            if data_tabel[row, 'MONTH'] == month:
                capacity = data_tabel[row, 'SEATS']
                passengers = data_tabel[row, 'PASSENGERS']
                if capacity != 0:
                    fullness = passengers/capacity
                    mweight = data_tabel[row, 'DEPARTURES_PERFORMED']/(month_weights[i]*total_dep_perf)
                    fullness_mw = mweight*fullness
                    tot_month_fullness += fullness_mw
                    tot_fullness_percent = 100*tot_month_fullness
        month_fullness.append(tot_fullness_percent)
        
    months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
    latestmonth = 9

    traffic_summary_tabel = tabel.Tabel({"Month": months[:latestmonth], 
                                         "Departures Performed(100k)": month_departures_perf_100k, 
                                         "Passengers(100k)": month_passengers_100k, 
                                         "Fullness(%)": month_fullness})
    print(traffic_summary_tabel)
    
    show_graphs = input("Would you like to see this table visualized with graphs? (yes/no) ")
    if show_graphs.lower() == 'no':
        sys.exit("Exploration for this variable and variables similar has ended.")
    if show_graphs.lower() == 'yes':
        d = figure(title="Departures Performed during Coronavirus",
                   x_range = traffic_summary_tabel['Month'],
                   x_axis_label = "Month", 
                   y_axis_label = "# Departures (100,000s)")
        d.line(x=traffic_summary_tabel['Month'], y=traffic_summary_tabel['Departures Performed(100k)'], color="green")

        p = figure(title="Total Passengers during Coronavirus",
                   x_range = traffic_summary_tabel['Month'],
                   x_axis_label = "Month", 
                   y_axis_label = "# Passengers (100,000s)")
        p.line(x=traffic_summary_tabel['Month'], y=traffic_summary_tabel['Passengers(100k)'], color="blue")

        f = figure(title="Flight Fullness during Coronavirus",
                   x_range = traffic_summary_tabel['Month'],
                   x_axis_label = "Month", 
                   y_axis_label = "Fullness (%)")
        f.line(x=traffic_summary_tabel['Month'], y=traffic_summary_tabel['Fullness(%)'], color="orange")
        
        show(column(d,p,f))

 Month     |   Departures Performed(100k) |   Passengers(100k) |   Fullness(%)
-----------+------------------------------+--------------------+---------------
 January   |                      7.56527 |           622.592  |      68.4798
 February  |                      7.12561 |           604.347  |      70.4165
 March     |                      6.68499 |           347.539  |      43.0334
 April     |                      2.67887 |            29.2477 |       9.91042
 May       |                      2.67347 |            83.8789 |      27.7601
 June      |                      3.28527 |           164.582  |      43.3879
 July      |                      4.69808 |           231.59   |      42.527
 August    |                      4.93041 |           242.296  |      42.1665
 September |                      4.36775 |           255.168  |      52.7028
9 rows ['<U9', '<f8', '<f8', '<f8']
Would you like to see this table visualized with graphs? (yes/no) yes


In [30]:
if var_chosen.upper() == 'PAYLOAD' or var_chosen.upper() == 'FREIGHT' or var_chosen.upper() == 'MAIL':

    month_cargo_fill = []
    month_freight = []
    month_mail = []

    for i in range(max(data_tabel['MONTH'])):
        month = i+1
        tot_freight = 0
        tot_mail = 0

        for row in range(len(data_tabel)): 
            if data_tabel[row, 'MONTH'] == month:
                tot_freight += np.float64(data_tabel[row, 'FREIGHT'])
                tot_mail += data_tabel[row, 'MAIL']
                
        month_freight.append(tot_freight)
        month_mail.append(tot_mail)
      
    for i in range(max(data_tabel['MONTH'])):
        month = i+1
        tot_month_fill = 0

        for row in range(len(data_tabel)): 
            if data_tabel[row, 'MONTH'] == month:
                freight = data_tabel[row, 'FREIGHT']
                mail = data_tabel[row, 'MAIL']
                payload = data_tabel[row, 'PAYLOAD']
                if data_tabel[row, 'AIRCRAFT_CONFIG'] != 1 and data_tabel[row, 'PAYLOAD'] != 0:
                    cargo_filled = (freight+mail)/payload
                    mweight = data_tabel[row, 'DEPARTURES_PERFORMED']/(month_weights[i]*total_dep_perf)
                    fill_mw = mweight*cargo_filled
                    tot_month_fill += fill_mw
                    tot_fill_percent = 100*tot_month_fill
        
        month_cargo_fill.append(tot_fill_percent)                             
                                
    months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
    latestmonth = 9

    cargo_summary_tabel = tabel.Tabel({"Month": months[:latestmonth], 
                                       "Total Freight(lbs.)": month_freight, 
                                       "Total Mail(lbs.)": month_mail, 
                                       "Payload filled by Cargo(%)": month_cargo_fill})
    print(cargo_summary_tabel)

    show_graphs = input("Would you like to see this table visualized with graphs? (yes/no) ")
    if show_graphs.lower() == 'no':
        sys.exit("Exploration for this variable and variables similar has ended.")
    if show_graphs.lower() == 'yes':
        fr = figure(title="Freight Transported during Coronavirus",
                    x_range = cargo_summary_tabel['Month'],
                    x_axis_label = "Month", 
                    y_axis_label = "Pounds of Freight")
        fr.line(x=cargo_summary_tabel['Month'], y=cargo_summary_tabel['Total Freight(lbs.)'], color="red")

        m = figure(title="Mail Transported during Coronavirus",
                   x_range = cargo_summary_tabel['Month'],
                   x_axis_label = "Month", 
                   y_axis_label = "Pounds of Mail")
        m.line(x=cargo_summary_tabel['Month'], y=cargo_summary_tabel['Total Mail(lbs.)'], color="purple")

        c = figure(title="Total Cargo as Percentage of Payload during Coronavirus",
                   x_range = cargo_summary_tabel['Month'],
                   x_axis_label = "Month", 
                   y_axis_label = "% of Payload filled by Cargo")
        c.line(x=cargo_summary_tabel['Month'], y=cargo_summary_tabel['Payload filled by Cargo(%)'], color="cyan")

        show(column(fr,m,c))

 Month     |   Total Freight(lbs.) |   Total Mail(lbs.) |   Payload filled by Cargo(%)
-----------+-----------------------+--------------------+------------------------------
 January   |           2.08664e+09 |        1.02828e+08 |                      2.83488
 February  |           1.87144e+09 |        9.36947e+07 |                      2.7176
 March     |           2.18413e+09 |        9.30821e+07 |                      3.13963
 April     |           2.17271e+09 |        7.85074e+07 |                      7.92949
 May       |           2.32832e+09 |        8.28912e+07 |                      8.72329
 June      |           2.37215e+09 |        9.49798e+07 |                      7.38114
 July      |           2.46356e+09 |        1.08833e+08 |                      5.30026
 August    |           2.33692e+09 |        1.07928e+08 |                      4.85301
 September |           2.40933e+09 |        1.12872e+08 |                      5.60551
9 rows ['<U9', '<f8', '<i4', '<f8']
Would y

In [31]:
if var_chosen.upper() in data_tabel.columns[6:]:
    print('Sorry, insights have not yet been made for these kinds of variables.')